In [1]:
import logging;logging.getLogger().setLevel(logging.INFO)
import pickle,gc
import bioplatform as bp
from bioplatform import providers,files
from bioplatform.job import Job
import bioplatform.python as py


def launder(obj):
    """Serialize and deserialize an object"""
    return pickle.loads( pickle.dumps(obj))
#reload(du);reload(files);reload(api)

<h3>Basic api functions</h3>

In [2]:
"Test subprocess file staging"
provider = providers.Subprocess()
input_files = {'myfile.txt': files.StringContainer('abc123','tempname') }
job = Job(provider,'platform/base_image','cp -v myfile.txt newfile.txt && echo test!',
          inputs=input_files)
job.wait()
assert 'newfile.txt' in job.get_output()
assert job.stdout.strip().split('\n')[-1] == 'test!'
assert job.get_output('newfile.txt').contents.strip() == 'abc123'

In [3]:
"""Test docker image building (requires a docker-machine named "default")"""
import docker
from bioplatform import docker_utils as du

#Create a provisioned image
wdir = '/mytmp'
inputs = {'a.pdf':files.LocalFile('test.pdf')}
client = du.docker_machine_client('default')
test_build_context = du.create_build_context('phusion/baseimage',inputs,wdir)
assert 'Dockerfile' in test_build_context
assert 'inputs/a.pdf' in test_build_context

#Check that the container was provisioned and runs succesfully
imageid = du.create_provisioned_image(client,'phusion/baseimage',wdir,inputs)
container = client.create_container(imageid,command='ls /mytmp')
client.start(container)
client.wait(container)
assert client.logs(container,stdout=True,stderr=False).strip() == 'a.pdf'
assert client.logs(container,stdout=False,stderr=True).strip() == ''

INFO:requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): 192.168.99.100
INFO:root:building image:{"stream":"Step 1 : FROM phusion/baseimage\n"}
INFO:root:building image:{"stream":" ---\u003e ea7f812691cd\n"}
INFO:root:building image:{"stream":"Step 2 : COPY inputs /mytmp\n"}
INFO:root:building image:{"stream":" ---\u003e d01985ba60e8\n"}
INFO:root:building image:{"stream":"Removing intermediate container c6ffb2c310de\n"}
INFO:root:building image:{"stream":"Successfully built d01985ba60e8\n"}


In [4]:
"""Test docker jobs"""
client = du.docker_machine_client('default')
provider = providers.Docker(client)
input_files = {'myfile.txt': files.StringContainer('abc123','tempname') }
job = Job(provider,'phusion/baseimage',
                    'cp -v myfile.txt newfile.txt && echo test!', inputs=input_files)
job.wait()
assert 'newfile.txt' in job.get_output()
assert job.stdout.strip().split('\n')[-1] == 'test!'
assert job.get_output('newfile.txt').contents.strip() == 'abc123'

INFO:requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): 192.168.99.100
INFO:root:building image:{"stream":"Step 1 : FROM phusion/baseimage\n"}
INFO:root:building image:{"stream":" ---\u003e ea7f812691cd\n"}
INFO:root:building image:{"stream":"Step 2 : COPY inputs /default_wdir\n"}
INFO:root:building image:{"stream":" ---\u003e Using cache\n"}
INFO:root:building image:{"stream":" ---\u003e a631ec7530e4\n"}
INFO:root:building image:{"stream":"Successfully built a631ec7530e4\n"}
INFO:requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): 192.168.99.100


<h3>Launchers</h3>

In [5]:
"Subprocess launcher"
proc = providers.Subprocess()
job = proc.launch(None,'echo my path is %s' % bp.bioplatform_path)
job.wait()
assert job.stdout.strip() == 'my path is %s' % bp.bioplatform_path
assert job.stderr == ''

In [6]:
"Docker launcher"
proc = providers.Docker( du.docker_machine_client() )
job = proc.launch('phusion/baseimage','echo my path is %s' % bp.bioplatform_path)
job.wait()
assert job.stdout.strip() == 'my path is %s' % bp.bioplatform_path
assert job.stderr == ''

INFO:requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): 192.168.99.100
INFO:root:building image:{"stream":"Step 1 : FROM phusion/baseimage\n"}
INFO:root:building image:{"stream":" ---\u003e ea7f812691cd\n"}
INFO:root:building image:{"stream":"Step 2 : RUN mkdir -p /default_wdir\n"}
INFO:root:building image:{"stream":" ---\u003e Using cache\n"}
INFO:root:building image:{"stream":" ---\u003e 5c4855d21330\n"}
INFO:root:building image:{"stream":"Successfully built 5c4855d21330\n"}


In [12]:
"Python launcher"
docker_provider = providers.Docker( du.docker_machine_client() )
pylauncher = py.PythonLauncher(docker_provider,'minimal_python')
y=10
def increment_by(x): return x+y
job = pylauncher(increment_by,3)
job.wait()
assert job.result==13

INFO:requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): 192.168.99.100
INFO:root:building image:{"stream":"Step 1 : FROM minimal_python\n"}
INFO:root:building image:{"stream":" ---\u003e 085e74ad7559\n"}
INFO:root:building image:{"stream":"Step 2 : COPY inputs /default_wdir\n"}
INFO:root:building image:{"stream":" ---\u003e ff5272862036\n"}
INFO:root:building image:{"stream":"Removing intermediate container 411150f439fd\n"}
INFO:root:building image:{"stream":"Successfully built ff5272862036\n"}
INFO:requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): 192.168.99.100


<h4>Python tests</h4>

In [ ]:
"""Test python function packaging"""
#TODO: see if we can blow away the namespace before running the laundered package
import pickle as cp
#reload(py)

def increment(x): return x+1
pkg = py.PackagedFunction(increment,32)
laundered_pkg = launder(pkg)
assert laundered_pkg.run(increment) == 33

In [ ]:
"""Test instance method packaging"""
class MyClass(object):
    def __init__(self,x):
        self.x = x
    def inc_x(self):
        self.x += 1
        return "Changed x: %d -> %d"%(self.x-1,self.x)
    
def make_serialized_object():
    myinstance = MyClass(5)
    pkg = py.PackagedFunction(myinstance.inc_x)
    return pickle.dumps(pkg)

pickled_pkg = make_serialized_object()
MyClass_hide = MyClass
gc.collect()
pkg = pickle.loads(pickled_pkg)

assert pkg.run() == 'Changed x: 5 -> 6'
assert pkg.obj.x == 6

In [ ]:
"""Test function packaging with closure variables"""
y=10
def increment_by(x): return x+y
pkg = py.PackagedFunction(increment_by,32)
laundered_pkg = launder(pkg)
assert laundered_pkg.run(increment_by) == 42

In [ ]:
"""Test function packaging with closure module"""
import math
def calculate_sin(x): return math.sin(x)
pkg = py.PackagedFunction(calculate_sin,math.pi/2.0)
laundered_pkg = launder(pkg)
assert abs(laundered_pkg.run(calculate_sin) - 1.0) < 1.e-15

In [ ]:
"""Test function packaging with two nested functions"""
def add_one(x): return x+1
def add_two(x): return add_one(add_one(x))
pkg = py.PackagedFunction(add_two,3)
laundered_pkg = launder(pkg)
assert laundered_pkg.run(add_two) == 5

In [ ]:
"""Test python job objects"""
client = du.docker_machine_client('default')
docker_provider = providers.Docker(client)

class FClass(object):
    def __init__(self,x):
        self.x = x
    def inc_x(self):
        self.x += 1.0
        return "Changed x: %f -> %f"%(self.x-1.0,self.x)
instance = FClass(10.3)
fcall = py.PythonCall(instance.inc_x,[],{})

subproc_job = py.PythonJob(providers.Subprocess(),None,fcall)
docker_job = py.PythonJob(docker_provider,'phusion/baseimage',fcall)


In [ ]:
job.exception